# How to Use NumPy Semantics in MXNet with `mxnet.numpy` Module

## NumPy Shape Semantics

### Example 

| Shape Example  | MXNet (before)  | MXNet/NumPy   |
|:---:|:---:|:---:|
| `()`   | unknown  | Scalar tensor   |
| `(2, 0, 1)` | Second dimension unknown | Zero-size tensor |
| `None`(Python) | N/A | Unknown |
| `(2, -1, 0)`(C++) | N/A | Second dim uknown|

### Affected modules
- Shape inference: imperative, symbolic, Gluon
- Legacy operators (not recommended to use)
- MXNet/NumPy operators

## NumPy Array Semantics
**Definition:** The type of created ndarrays is `mxnet.numpy.ndarray`/`mxnet.symbol.numpy._Symbol`, instead of `mxnet.ndarray.NDArray`/`mxnet.symbol.Symbol` (only affects Gluon modules).
- Block/HybridBlock
    - Parameter creation and initialization.
    - Inputs/outputs (symbol/ndarray) of `__call__`/`forward`/`hybrid_forward`.
    - Computational graph construction.
- Dataloader

## Dependency of Two Types of Semantics
- It is required to keep NumPy shape semantics active while activating NumPy array semantics.
- Deactivating NumPy shape semantics while NumPy array semantics is still active is not allowed.

In [ ]:
import logging
import mxnet as mx
from mxnet import np, npx, gluon

logging.basicConfig(level=logging.INFO)

try:
    npx.set_np(shape=False, array=True)
except ValueError as e:
    print(e)

## How to Enable NumPy Shape semantics

In [ ]:
try:
    a = mx.nd.random.uniform(shape=())
except mx.MXNetError as e:
    print(e)

In [ ]:
try:
    b = mx.nd.random.uniform(shape=(2, 0, 1))
except mx.MXNetError as e:
    print(e)

In [ ]:
try:
    c = np.random.uniform()
except mx.MXNetError as e:
    print(e)

In [ ]:
try:
    d = np.random.uniform(size=(2, 0, 1))
except mx.MXNetError as e:
    print(e)  

In [ ]:
npx.set_np(shape=True, array=False)

In [ ]:
a = mx.nd.random.uniform(shape=())
b = mx.nd.random.uniform(shape=(2, 0, 1))
c = np.random.uniform()
d = np.random.uniform(size=(2, 0, 1))

print('type(a) =', type(a))
print('a.shape = ', a.shape)
print('a.size = ', a.size)

print('type(b) =', type(b))
print('b.shape = ', b.shape)
print('b.size = ', b.size)

print('type(c) =', type(c))
print('c.shape = ', c.shape)
print('c.size = ', c.size)

print('type(d) =', type(d))
print('d.shape = ', d.shape)
print('d.size = ', d.size)

## How to Enable NumPy Array Semantics

### Parameters

In [ ]:
npx.reset_np()  # reset two types of semantics to the default state, which is False for both of them

from mxnet.gluon import nn
class Net(gluon.Block):
    def __init__(self, in_units=0, **kwargs):  # 0 means in_units is unknown and must be inferred at runtime
        super(Net, self).__init__(**kwargs)
        with self.name_scope():
            self.dense0 = nn.Dense(5, in_units=in_units)
            self.dense1 = nn.Dense(5, in_units=in_units)
            
    def forward(self, x):
        return self.dense1(self.dense0(x))

net1 = Net()
net1.initialize()
net1(mx.nd.zeros((3, 10)))
for k, v in net1.collect_params().items():
    print('parameter {}, type {}'.format(k, str(type(v.data()))))

In [ ]:
npx.set_np()

In [ ]:
net2 = Net()
net2.initialize()
net2(np.zeros((3, 10)))
for k, v in net2.collect_params().items():
    print('parameter {}, type {}'.format(k, str(type(v.data()))))

### Dataloader

In [ ]:
import sys
import os
from mxnet.gluon import data as gdata


npx.reset_np()


def load_data_fashion_mnist(batch_size, resize=None, root=os.path.join(
        '~', '.mxnet', 'datasets', 'fashion-mnist')):
    """Download the Fashion-MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)]
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)

    mnist_train = gdata.vision.FashionMNIST(root=root, train=True)
    mnist_test = gdata.vision.FashionMNIST(root=root, train=False)
    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

In [ ]:
train_iter, test_iter = load_data_fashion_mnist(16)

for X, y in train_iter:
    print('type(X) = ', type(X))
    print('type(y) = ', type(y))
    break

In [ ]:
npx.set_np()

In [ ]:
train_iter, test_iter = load_data_fashion_mnist(16)

for X, y in train_iter:
    print('type(X) = ', type(X))
    print('type(y) = ', type(y))
    break